# Flotorch ADK - Flotorch Memory Service Example

This notebook demonstrates how to integrate Flotorch Memory Service with ADK agents.
Shows how to store and retrieve conversations using Flotorch Memory Service.

## Key Features
- Flotorch Memory Service integration
- Automatic conversation storage  
- Memory-aware agent responses

## Requirements
- FLOTORCH_API_KEY environment variable
- FLOTORCH_BASE_URL environment variable


In [ ]:
# Setup and imports
import asyncio
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Environment setup complete")


✓ Environment setup complete


In [3]:
# Flotorch ADK imports
from flotorch.adk.llm import FlotorchADKLLM
from flotorch.adk.memory import FlotorchMemoryService
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk import Runner
from google.genai import types
from google.adk.tools import load_memory

print("✓ Flotorch ADK imports successful")


✓ Flotorch ADK imports successful


In [4]:
# Configuration
APP_NAME = "flotorch_memory_service_example"
USER_ID = "flotorch_user_123"
MEMORY_PROVIDER = "memo-provider"

print(f"App Name: {APP_NAME}")
print(f"User ID: {USER_ID}")
print(f"Memory Provider: {MEMORY_PROVIDER}")


App Name: flotorch_memory_service_example
User ID: flotorch_user_123
Memory Provider: memo-provider


In [5]:
# Create Flotorch LLM model
model = FlotorchADKLLM(
    model_id="openai/gpt-4o-mini",
    api_key=os.getenv("FLOTORCH_API_KEY"),
    base_url=os.getenv("FLOTORCH_BASE_URL"),
)

print("✓ Flotorch LLM model created")


✓ Flotorch LLM model created


In [6]:
# Initialize Flotorch Memory Service
flotorch_memory_service = FlotorchMemoryService(
    name=MEMORY_PROVIDER,
    api_key=os.getenv("FLOTORCH_API_KEY"),
    base_url=os.getenv("FLOTORCH_BASE_URL"),
)

print("✓ Flotorch Memory Service initialized")


✓ Flotorch Memory Service initialized


In [7]:
# Create Flotorch agent with memory tool
agent = LlmAgent(
    name="Flotorch_Memory_Assistant",
    description="A Flotorch AI assistant with Flotorch Memory Service integration",
    instruction="You are Flotorch AI assistant with memory capabilities. Use your memory to provide contextual responses.",
    model=model,
    tools=[load_memory],  # Memory tool for accessing Flotorch Memory Service
)

print("✓ Flotorch agent with memory tool created")


✓ Flotorch agent with memory tool created


In [8]:
# Setup runner with Flotorch Memory Service
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=flotorch_memory_service  # Flotorch Memory Service integration
)

print("✓ Runner setup with memory service complete")


✓ Runner setup with memory service complete


In [9]:
# Helper function to send messages
def run_single_turn(query, session_id, user_id):
    """Send message to Flotorch agent with memory service support."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that."

print("✓ Helper function defined")


✓ Helper function defined


In [10]:
# Create session and test memory-enabled agent
async def test_memory_agent():
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"Flotorch agent with Memory Service ready! Session: {session.id}")
    print("The agent can now access and store memories using Flotorch Memory Service")
    
    # Test with a simple message
    test_message = "Hello! My name is John and I work as a software engineer."
    response = run_single_turn(test_message, session.id, USER_ID)
    
    print(f"\nUser: {test_message}")
    print(f"Flotorch AI: {response}")
    
    return session

# Run the test
session = await test_memory_agent()


Flotorch agent with Memory Service ready! Session: 2c0216a9-1b1b-456b-bad4-28b97ea033bc
The agent can now access and store memories using Flotorch Memory Service

User: Hello! My name is John and I work as a software engineer.
Flotorch AI: Hello, John! It's great to meet you. I see that you work as a software engineer. By the way, do you have any pets?


In [11]:
# Interactive chat function with memory
def chat_with_memory(message):
    """Simple function to chat with the memory-enabled agent."""
    response = run_single_turn(message, session.id, USER_ID)
    print(f"You: {message}")
    print(f"Flotorch AI: {response}")
    return response

# Have a conversation that demonstrates memory
print("=== Conversation with Memory ===")
chat_with_memory("I love hiking and have a pet dog named Max.")

print("\n" + "-"*50)
chat_with_memory("What's my job?")

print("\n" + "-"*50)
chat_with_memory("What's my dog's name?")


=== Conversation with Memory ===
You: I love hiking and have a pet dog named Max.
Flotorch AI: That's awesome! Hiking is a great way to enjoy nature, and having a dog like Max must make it even more fun. Just to clarify, I have a note that you have a dog named Rocky. Is there a mix-up, or do you have two dogs?

--------------------------------------------------
You: What's my job?
Flotorch AI: You work as a software engineer.

--------------------------------------------------
You: What's my dog's name?
Flotorch AI: You mentioned that your dog's name is Max, but I also have a note that you have a dog named Rocky. Could you clarify if you have two dogs or if there's been a mix-up?


"You mentioned that your dog's name is Max, but I also have a note that you have a dog named Rocky. Could you clarify if you have two dogs or if there's been a mix-up?"

In [12]:
# Store conversation in Flotorch Memory Service
async def store_conversation():
    print("Storing conversation in Flotorch Memory Service...")
    
    completed_session = await runner.session_service.get_session(
        app_name=APP_NAME, 
        user_id=USER_ID, 
        session_id=session.id
    )
    
    await flotorch_memory_service.add_session_to_memory(completed_session)
    print("✓ Conversation successfully stored in Flotorch Memory Service!")

# Store the conversation
await store_conversation()


Storing conversation in Flotorch Memory Service...
✓ Conversation successfully stored in Flotorch Memory Service!


In [13]:
# Test memory recall with new session
async def test_memory_recall():
    new_session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"New session started: {new_session.id}")
    print("Testing memory recall...")
    
    # Test if agent remembers previous conversation
    recall_message = "Do you remember what I told you about my job and pet?"
    response = run_single_turn(recall_message, new_session.id, USER_ID)
    
    print(f"\nUser: {recall_message}")
    print(f"Flotorch AI: {response}")
    
    return new_session

# Test memory recall
new_session = await test_memory_recall()


New session started: a5c6eb33-34d6-44e3-a51a-c2845f5a97a9
Testing memory recall...

User: Do you remember what I told you about my job and pet?
Flotorch AI: You mentioned that your name is John, and you work as a software engineer. You also have a pet dog named Max, and you love hiking.


## Summary

This notebook demonstrated:
- ✓ Setting up Flotorch Memory Service
- ✓ Creating memory-enabled agents with load_memory tool
- ✓ Storing conversations in Flotorch Memory Service
- ✓ Memory recall across different sessions
- ✓ Building memory-aware AI applications

You can now use `chat_with_memory("your message")` to continue the conversation with memory!
